In [22]:
import lightgbm as lgb

from sklearn import datasets
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import gc

In [23]:
# data load
test_meta = pd.read_csv('../input/test_set_metadata.csv')
best_sub = pd.read_csv('../output/predictions_1210_v5.csv') #0.957 no val
best_2_sub = pd.read_csv('../output/predictions_1216_v2.csv') #0.935 no tuning
best_3_sub = pd.read_csv('../output/predictions_1216_v5.csv') #0.971 simple
best_4_sub = pd.read_csv('../output/predictions_1217_v1.csv') #0.923 Best!
sub_1 = pd.read_csv('../input/subm_0.619987_2018-12-07-15-11.csv')
sub_2 = pd.read_csv('../input/subm_0.751366_2018-12-10-14-48.csv')
specz = pd.read_pickle('../input/preds_result_20181217_specz.pickle')
NN = pd.read_csv('../output/predictions_1216_v5.csv')

In [24]:
# setting
total_cols = ["object_id","class_6","class_15","class_16","class_42","class_52","class_53","class_62","class_64","class_65","class_67","class_88","class_90","class_92","class_95","class_99"]
best_sub = best_sub.sort_values('object_id').reset_index()
best_sub = best_sub[total_cols]
sub_id = best_sub['object_id']

NN = NN.sort_values('object_id').reset_index(drop=True)
NN = NN[total_cols]

best_2_sub = best_2_sub.sort_values('object_id').reset_index(drop=True)
best_2_sub = best_2_sub[total_cols]

best_3_sub = best_3_sub.sort_values('object_id').reset_index(drop=True)
best_3_sub = best_3_sub[total_cols]

best_4_sub = best_4_sub.sort_values('object_id').reset_index(drop=True)
best_4_sub = best_4_sub[total_cols]

sub_1 = sub_1.sort_values('object_id').reset_index(drop=True)
sub_1 = sub_1[total_cols]

specz = specz[total_cols]

In [25]:
#sub_2はダブりがあるので別処理
sub_2 = sub_2[~sub_2['object_id'].duplicated()]
sub_2 = sub_2.sort_values('object_id').reset_index(drop=True)
sub_2 = sub_2[total_cols]

In [26]:
# 確認
best_4_sub.tail(3)

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492887,130787974,0.0,0.011920,0.0,0.938404,0.006439,0.0,0.033704,0.000632,0.0,0.001751,0.000664,0.002506,0.0,0.000694,0.033586
3492888,130788053,0.0,0.710568,0.0,0.081806,0.008449,0.0,0.015320,0.170524,0.0,0.001297,0.000738,0.006685,0.0,0.001130,0.123237
3492889,130788054,0.0,0.149767,0.0,0.442504,0.069893,0.0,0.168056,0.003935,0.0,0.017299,0.003590,0.126638,0.0,0.003391,0.178003


In [27]:
best_sub.tail(3)

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492887,130787974,0.0,0.055572,0.0,0.865361,0.010412,0.0,0.052156,0.000753,0.0,0.002588,0.000821,0.006375,0.0,0.001978,0.064322
3492888,130788053,0.0,0.600019,0.0,0.215518,0.050707,0.0,0.031279,0.063500,0.0,0.007616,0.001239,0.022541,0.0,0.001962,0.142457
3492889,130788054,0.0,0.040561,0.0,0.427404,0.265005,0.0,0.081027,0.003802,0.0,0.036184,0.004323,0.121593,0.0,0.004117,0.167444


In [28]:
# 0.913
'''
merge_sub = (best_4_sub.iloc[:,1:] * 0.5 + 
             best_3_sub.iloc[:,1:] * 0.1 +
             best_2_sub.iloc[:,1:] * 0.1 +
             best_sub.iloc[:,1:] * 0.1 +
             sub_1.iloc[:,1:] * 0.08 + 
             sub_2.iloc[:,1:] * 0.08 + 
             NN.iloc[:,1:] * 0.04 )
merge_sub = pd.merge(sub_id.to_frame(),merge_sub,left_index = True, right_index=True,how='left')
'''

"\nmerge_sub = (best_4_sub.iloc[:,1:] * 0.5 + \n             best_3_sub.iloc[:,1:] * 0.1 +\n             best_2_sub.iloc[:,1:] * 0.1 +\n             best_sub.iloc[:,1:] * 0.1 +\n             sub_1.iloc[:,1:] * 0.08 + \n             sub_2.iloc[:,1:] * 0.08 + \n             NN.iloc[:,1:] * 0.04 )\nmerge_sub = pd.merge(sub_id.to_frame(),merge_sub,left_index = True, right_index=True,how='left')\n"

In [29]:
# last_submit
merge_sub = (best_4_sub.iloc[:,1:] * 0.7 + 
             best_3_sub.iloc[:,1:] * 0.05 +
             #best_2_sub.iloc[:,1:] * 0.1 +
             #best_sub.iloc[:,1:] * 0.1 +
             sub_1.iloc[:,1:] * 0.1 + 
             sub_2.iloc[:,1:] * 0.1 + 
             NN.iloc[:,1:] * 0.05 )
merge_sub = pd.merge(sub_id.to_frame(),merge_sub,left_index = True, right_index=True,how='left')

In [8]:
# merge

#merge_sub = (best_sub.iloc[:,1:] * 0.75 + sub_1.iloc[:,1:] * 0.125 + + sub_2.iloc[:,1:] * 0.125)
#merge_sub = (best_sub.iloc[:,1:] * 0.5 + best_2_sub.iloc[:,1:] * 0.3 + sub_1.iloc[:,1:] * 0.1 + sub_2.iloc[:,1:] * 0.1) #0.939
#merge_sub = (best_4_sub.iloc[:,1:] * 0.75 + best_3_sub.iloc[:,1:] * 0.08 + sub_1.iloc[:,1:] * 0.07 + sub_2.iloc[:,1:] * 0.07 + NN.iloc[:,1:] * 0.03 ) #
#merge_sub = pd.merge(sub_id.to_frame(),merge_sub,left_index = True, right_index=True,how='left')

In [30]:
#specz_prod
specz_id = specz.object_id
merge_specz = merge_sub[merge_sub['object_id'].isin(specz['object_id'])].reset_index(drop=True)
merge_not_specz_sub = merge_sub[~merge_sub['object_id'].isin(specz['object_id'])]

In [31]:
merge_specz_sub = (specz.iloc[:,1:] * 0.03 + merge_specz.iloc[:,1:] * 0.97) #
merge_specz_sub = pd.merge(specz_id.to_frame(),merge_specz,on='object_id',how='left')

In [32]:
merge_sub = merge_specz_sub.append(merge_not_specz_sub).sort_values('object_id')

In [33]:
merge_sub.head(3)

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
0,13,0.000022,0.002105,0.000012,0.731146,0.174048,0.000049,0.067880,0.001080,0.000010,0.002060,0.001236,0.013398,0.000019,0.001507,0.106921
1,14,0.000023,0.011993,0.000022,0.161036,0.171703,0.000053,0.127910,0.002565,0.000178,0.026252,0.002376,0.478971,0.000025,0.006162,0.165024
2,17,0.000052,0.030568,0.000054,0.067707,0.051814,0.000104,0.027196,0.005589,0.000074,0.031810,0.004345,0.764750,0.000030,0.008181,0.103652


In [34]:
merge_sub.tail(3)

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492887,130787974,0.000086,0.020783,0.000012,0.909612,0.006956,0.000056,0.053840,0.000509,0.000016,0.001540,0.000604,0.002557,0.000020,0.000837,0.046946
3492888,130788053,0.000031,0.675961,0.000051,0.111151,0.015644,0.000070,0.017987,0.157886,0.000135,0.002015,0.000661,0.014588,0.000036,0.000990,0.131080
3492889,130788054,0.000020,0.147540,0.000018,0.429242,0.127227,0.000048,0.140574,0.003090,0.000031,0.015403,0.002843,0.119606,0.000026,0.002755,0.177954


In [35]:
exgal_cols = ["class_15","class_42","class_52","class_62","class_64","class_67","class_88","class_90","class_95"]
gal_cols = ["class_6","class_16","class_53","class_65","class_92"]
total_cols = ["class_6","class_15","class_16","class_42","class_52","class_53","class_62","class_64","class_65","class_67","class_88","class_90","class_92","class_95"]

In [36]:
# post process
pred_hostgal = pd.merge(merge_sub,test_meta[['object_id','hostgal_photoz']],on='object_id',how='left')

pred_gal = pred_hostgal[pred_hostgal['hostgal_photoz']==0].copy()
pred_gal[exgal_cols]=0

pred_ex_gal = pred_hostgal[pred_hostgal['hostgal_photoz']!=0].copy()
pred_ex_gal[gal_cols]=0

pred = pred_gal.append(pred_ex_gal)

In [37]:
del pred['hostgal_photoz']
result = pred.set_index('object_id').sort_values('object_id')

In [38]:
result.head(3)

,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
object_id,,,,,,,,,,,,,,,
13,0.0,0.002105,0.0,0.731146,0.174048,0.0,0.067880,0.001080,0.0,0.002060,0.001236,0.013398,0.0,0.001507,0.106921
14,0.0,0.011993,0.0,0.161036,0.171703,0.0,0.127910,0.002565,0.0,0.026252,0.002376,0.478971,0.0,0.006162,0.165024
17,0.0,0.030568,0.0,0.067707,0.051814,0.0,0.027196,0.005589,0.0,0.031810,0.004345,0.764750,0.0,0.008181,0.103652


In [39]:
result.to_csv('../output/predictions_final_submit.csv', float_format='%.6f')

In [40]:
import os
os.chdir('/home/makoto.hyodo/')

In [41]:
%%bash
sh kaggle_submit_fix.sh PLAsTiCC output predictions_final_submit.csv PLAsTiCC-2018 final_submit

Successfully submitted to PLAsTiCC Astronomical Classification

: not found
100%|██████████| 480M/480M [00:23<00:00, 21.5MB/s]


In [42]:
os.chdir('/home/makoto.hyodo/kaggle/PLAsTiCC/script')